In [ ]:
# import libraries
import os
import csv
from datetime import datetime

In [ ]:
from common.tables import PprRawAll
from common.base import session
from sqlalquemy import text

In [ ]:
# Settings
base_path = os.path.abspath(__file__ + "/../../")

In [ ]:
# START - Paths for new February 2021 data available

In [ ]:
# Raw path where we want to extract the new CSV data
raw_path = f"{base_path}/data/raw/downloaded_at=2021-02-01/ppr-all.csv"

In [ ]:
# END - Paths for new February 2021 data available

In [ ]:
def transform_case(input_string):
    """
    Lowercase string fields
    """
    return input_string.lower()

In [ ]:
def update_date_of_sale(date_input):
    """
    Update date format from DD/MM/YYYY to YYYY-MM-DD
    """
    current_format = datetime.strptime(date_input, "%d/%m/%Y")
    new_format = current_format.strftime("%Y-%m-%d")
    return new_format

In [ ]:
def update_description(description_input):
    """
    Simplify the description field for potentially future analysis, just return:
    - "new" if string contains "new" substring
    - "second-hand" if a string contains "second-hand" substring
    """
    description_input = transform_case(description_input)
    if "new" in description_input:
        return "new"
    elif "second-hand" in description_input:
        return "second-hand"
    return description_input